In [1]:
import sys
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as python_random
import tensorflow as tf
import seaborn as sns

sns.set_theme(color_codes=True)
import random
import joblib

sys.path.insert(0, ".././0.data-download/scripts/")
from data_loader import load_train_test_data, load_data
from matplotlib.pyplot import figure, gcf
from sklearn.decomposition import PCA
from tensorflow import keras

from keras.models import Model, Sequential

2023-07-19 14:18:03.476370: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
random.seed(18)
print(random.random())

0.18126486333322134


In [3]:
# load the data
data_directory = pathlib.Path("../0.data-download/data")
train_init, test_init, gene_stats = load_train_test_data(
    data_directory, train_or_test="all", load_gene_stats=True
)

data_dir = "../0.data-download/data/"
model_df, effect_df = load_data(data_dir, adult_or_pediatric="all")

In [4]:
# drop the string values
train_df = train_init.drop(columns=["ModelID", "age_and_sex"])
test_df = test_init.drop(columns=["ModelID", "age_and_sex"])

In [5]:
# subsetting the genes

# create dataframe containing the genes that passed an initial QC (see Pan et al. 2022) and their corresponding gene label and extract the gene labels
gene_dict_df = pd.read_parquet("../0.data-download/data/CRISPR_gene_dictionary.tsv", delimiter='\t')
gene_list_passed_qc = gene_dict_df.query("qc_pass").dependency_column.tolist()

# create new training and testing dataframes that contain only the corresponding genes
subset_train_df = train_df.filter(gene_list_passed_qc, axis=1)
subset_test_df = test_df.filter(gene_list_passed_qc, axis=1)

In [6]:
train_init["train_or_test"] = train_init.apply(lambda _: "train", axis=1)
test_init["train_or_test"] = test_init.apply(lambda _: "test", axis=1)

In [7]:
# load the latent dimension dataframe

latent_df = pd.read_parquet("../2.train-VAE/results/latent_df.parquet")

print(latent_df.shape)

latent_df.head(5)

(912, 50)


,ModelID,0,1,2,3,4,5,6,7,8,...,39,40,41,42,43,44,45,46,47,48
0,ACH-001549,-0.433536,-0.313002,0.106170,1.740107,0.051796,-0.092048,1.506002,-0.139188,1.185854,...,0.090623,-1.028285,0.596689,0.983311,2.057045,0.656402,0.335536,1.184920,0.809751,0.036602
1,ACH-001992,0.419395,0.565882,1.796919,-0.604782,1.892967,-0.170275,-0.377951,1.432515,-0.864623,...,1.425062,0.314461,2.994100,-0.368800,-0.018234,-1.465446,0.496746,0.427367,0.355352,1.249617
2,ACH-000035,0.732861,0.875819,-0.267914,1.066067,-0.391314,-0.501940,-1.188963,-0.868845,1.153341,...,0.535331,-2.002621,-0.183520,-0.247654,-0.957025,1.247603,1.202215,-0.729811,0.148200,0.842648
3,ACH-000911,0.678888,-0.606411,1.286772,-1.190569,-0.675025,1.643442,-0.169562,0.342549,-0.373929,...,0.133425,-0.908985,0.071771,-1.049606,1.250782,-0.244805,-0.322522,0.830723,-0.090198,-0.236837
4,ACH-000259,0.476313,0.367266,1.346829,0.106499,0.101344,0.814611,2.907445,0.997259,1.303916,...,-0.302948,1.337505,0.344960,1.044820,-0.602159,-0.835711,-1.958224,-0.292943,0.115363,1.033674


In [8]:
# create a data frame of both test and train gene effect data sorted by top 1000 highest gene variances
concat_frames = [train_init, test_init]
train_and_test = pd.concat(concat_frames).reset_index(drop=True)
train_and_test[["AgeCategory", "Sex"]] = train_and_test.age_and_sex.str.split(
    pat="_", expand=True
)
train_and_test_subbed = train_and_test.filter(gene_list_passed_qc, axis=1)
metadata_holder = []
metadata_holder = pd.DataFrame(metadata_holder)
metadata = metadata_holder.assign(
    ModelID=train_and_test.ModelID.astype(str),
    AgeCategory=train_and_test.AgeCategory.astype(str),
    Sex=train_and_test.Sex.astype(str),
    train_or_test=train_and_test.train_or_test.astype(str),
)
metadata

,ModelID,AgeCategory,Sex,train_or_test
0,ACH-001549,Adult,Male,train
1,ACH-001992,Adult,Male,train
2,ACH-000035,Adult,Male,train
3,ACH-000911,Adult,Male,train
4,ACH-000259,Pediatric,Female,train
...,...,...,...,...
907,ACH-000858,Adult,Male,test
908,ACH-000514,Adult,Male,test
909,ACH-000258,Adult,Female,test
910,ACH-002446,Adult,Male,test


In [9]:
AgeCategory = metadata.pop("AgeCategory")
Sex = metadata.pop("Sex")
train_test = metadata.pop("train_or_test")

In [10]:
# display clustered heatmap of coefficients
lut_pal = sns.cubehelix_palette(
    AgeCategory.unique().size, light=0.9, dark=0.1, reverse=True, start=1, rot=-2
)
put_pal = sns.cubehelix_palette(Sex.unique().size)
mut_pal = sns.color_palette("hls", train_test.unique().size)

lut = dict(zip(AgeCategory.unique(), lut_pal))
put = dict(zip(Sex.unique(), put_pal))
mut = dict(zip(train_test.unique(), mut_pal))

row_colors1 = AgeCategory.map(lut)
row_colors2 = Sex.map(put)
row_colors3 = train_test.map(mut)

network_node_colors = pd.DataFrame(row_colors1).join(
    pd.DataFrame(row_colors2).join(pd.DataFrame(row_colors3))
)

sns.set(font_scale=4.0)
g = sns.clustermap(
    latent_df.iloc[:,1:],
    method="ward",
    figsize=(10, 20),
    row_colors=network_node_colors,
    yticklabels=False,
    dendrogram_ratio=(0.1, 0.04),
    cbar_pos=(1, 0.3, 0.02, 0.6),
    cmap = "coolwarm",
)
g.ax_row_dendrogram.set_visible(False)
g.ax_col_dendrogram.set_visible(False)


xx = []
for label in AgeCategory.unique():
    x = g.ax_row_dendrogram.bar(0, 0, color=lut[label], label=label, linewidth=0)
    xx.append(x)
# add the legend
legend3 = plt.legend(
    xx,
    AgeCategory.unique(),
    loc="upper right",
    title="age category",
    ncol=1,
    bbox_to_anchor=(1.75, 0.87),
    bbox_transform=gcf().transFigure,
    fontsize = '30'
)


yy = []
for label in Sex.unique():
    y = g.ax_row_dendrogram.bar(0, 0, color=put[label], label=label, linewidth=0)
    yy.append(y)
# add the second legend
legend4 = plt.legend(
    yy,
    Sex.unique(),
    loc="upper right",
    title="sex",
    ncol=1,
    bbox_to_anchor=(1.75, 0.75),
    bbox_transform=gcf().transFigure,
    fontsize = '30'
)
plt.gca().add_artist(legend3)


zz = []
for label in train_test.unique():
    z = g.ax_row_dendrogram.bar(0, 0, color=mut[label], label=label, linewidth=0)
    zz.append(z)
# add the third legend
legend5 = plt.legend(
    zz,
    train_test.unique(),
    loc="upper right",
    title="train or test",
    ncol=1,
    bbox_to_anchor=(1.75, 0.6),
    bbox_transform=gcf().transFigure,
    fontsize = '30'
)
plt.gca().add_artist(legend4)


# save the figure
heat_save_path = pathlib.Path("../1.data-exploration/figures/heatmap.png")
plt.savefig(heat_save_path, bbox_inches="tight", dpi=600)

/home/ecohen/anaconda3/envs/gene_dependency_representations/lib/python3.10/site-packages/seaborn/matrix.py:560: UserWarning: Clustering large matrix with scipy. Installing `fastcluster` may give better performance.
  warnings.warn(msg)
/home/ecohen/anaconda3/envs/gene_dependency_representations/lib/python3.10/site-packages/seaborn/matrix.py:560: UserWarning: Clustering large matrix with scipy. Installing `fastcluster` may give better performance.
  warnings.warn(msg)


In [11]:
# load the weights dataframe

gene_weights_df = pd.read_parquet("../2.train-VAE/results/weight_matrix_encoder.parquet")

gene_weights_df

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,-0.038390,-0.010072,0.031175,-0.075533,0.147331,-0.008982,0.052985,-0.015245,-0.079500,0.008005,...,-0.036216,0.036293,-0.016212,-0.093938,0.030863,-0.003590,0.006323,0.007708,-0.020439,-0.008556
1,0.061055,-0.041610,0.013261,-0.021186,-0.000131,-0.074913,0.020564,0.069120,0.026609,-0.007725,...,0.004073,-0.006511,-0.023371,-0.005856,-0.021866,0.012609,0.005722,0.012721,-0.033651,0.012131
2,0.003446,-0.061648,0.016866,-0.082559,0.063057,-0.008855,-0.023239,-0.027252,-0.096719,0.083446,...,-0.027031,-0.039182,-0.053009,-0.093481,0.004291,-0.046227,0.006410,-0.070168,-0.013231,0.012959
3,-0.054095,0.004926,-0.055859,-0.010414,0.040705,0.030561,0.093116,-0.055641,-0.011852,0.020742,...,-0.008818,0.077715,0.003448,-0.049125,-0.054797,-0.080224,-0.024841,0.076140,-0.056244,-0.042099
4,-0.021163,0.025108,-0.054590,-0.031431,0.049623,-0.021309,-0.023290,-0.057167,-0.012089,-0.006595,...,-0.032671,-0.049985,-0.093279,-0.012201,-0.012458,-0.021575,0.008027,-0.047325,-0.039885,-0.052858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2727,-0.033738,-0.005072,-0.000603,-0.073180,0.027496,0.042648,0.067541,-0.056745,0.051380,-0.012439,...,0.007982,-0.034893,-0.005020,-0.023721,-0.013775,0.001517,0.001795,0.016650,-0.000199,-0.032603
2728,-0.074387,-0.018798,-0.006314,-0.043214,0.005455,-0.123786,0.054510,-0.108055,-0.066560,-0.010161,...,-0.080874,-0.058452,0.011608,0.016710,-0.007025,-0.050418,0.026847,0.017922,0.042893,-0.065465
2729,-0.062594,-0.009569,-0.016277,0.002783,0.039948,-0.111344,0.014660,-0.012605,0.027677,-0.072703,...,-0.048051,-0.037488,-0.039755,-0.135676,0.014519,-0.016897,-0.007807,0.022843,-0.002303,-0.010481
2730,-0.027604,-0.020581,0.045708,-0.069819,0.015183,-0.096062,-0.026753,0.013459,0.012703,-0.112076,...,-0.030383,-0.048858,-0.039914,-0.079369,-0.093655,0.003045,-0.001877,0.021659,-0.070840,0.057825


In [12]:
sns.clustermap(
    gene_weights_df,
    method="ward",
    figsize=(10, 20),
    #network_node_colors = pd.DataFrame(row_colors1).join(pd.DataFrame(row_colors2).join(pd.DataFrame(row_colors3))
    #row_colors=network_node_colors,
    yticklabels=False,
    dendrogram_ratio=(0.1, 0.04),
    cbar_pos=(1, 0.3, 0.02, 0.6),
    cmap = "coolwarm",
)


/home/ecohen/anaconda3/envs/gene_dependency_representations/lib/python3.10/site-packages/seaborn/matrix.py:560: UserWarning: Clustering large matrix with scipy. Installing `fastcluster` may give better performance.
  warnings.warn(msg)
/home/ecohen/anaconda3/envs/gene_dependency_representations/lib/python3.10/site-packages/seaborn/matrix.py:560: UserWarning: Clustering large matrix with scipy. Installing `fastcluster` may give better performance.
  warnings.warn(msg)
